In [ ]:
import pandas as pd
import os
import statsmodels.api as sm

In [ ]:
# Specify the directory where your files are located
data_dir = "/Users/jomarjordas/Documents/MSFIN299/MSFIN299-Research/_data/stockdata"

# Read in the index file
index_df = pd.read_csv(os.path.join(data_dir, 'psei.csv'))

# Calculate the daily returns for the index
index_df['returns'] = index_df['price'].pct_change()

# Loop through each company file
for ticker in ['ac']:
    # Read in the company file
    company_df = pd.read_csv(os.path.join(data_dir, f'{ticker}.csv'))

    # Drop any rows with missing values in the 'price' column
    company_df = company_df.dropna(subset=['price'])

    # Calculate the daily returns for the company, ignoring missing values
    try:
        company_df['returns'] = company_df['price'].pct_change(errors='ignore')
    except TypeError:
        print(f"Error: Could not calculate returns for {ticker}.csv file. Check for missing or invalid values in the 'price' column.")

    # Merge with index data
    merged_df = pd.merge(company_df, index_df, on='date', how='inner')

    # Calculate the excess returns
    merged_df['excess_return'] = merged_df['returns_x'] - merged_df['returns_y']

    # Calculate the expected return using CAPM
    X = sm.add_constant(merged_df['returns_y'])
    model = sm.OLS(merged_df['excess_return'], X)
    results = model.fit()
    beta = results.params[1]
    market_risk_premium = merged_df['returns_y'].mean() - 0.02 # Assume risk-free rate of 2%
    expected_return = 0.02 + beta * market_risk_premium

    # Add expected return column to company file
    merged_df['exp_return'] = expected_return

    # Write merged dataframe back to company file
    merged_df.to_csv(os.path.join(data_dir, f'{ticker}.csv'), index=False)


In [13]:
import pandas as pd
import yfinance as yf

def calculate_returns(ticker, start_date, end_date):
    # download data from Yahoo Finance
    df = yf.download(ticker, start=start_date, end=end_date)

    # calculate daily returns
    df['Return'] = df['Adj Close'].pct_change()

    # calculate rolling 30-day volatility
    df['Volatility'] = df['Return'].rolling(30).std() * (252 ** 0.5)

    # calculate rolling 30-day mean return
    df['Mean_Return'] = df['Return'].rolling(30).mean() * 252

    # calculate Sharpe Ratio
    df['Sharpe_Ratio'] = df['Mean_Return'] / df['Volatility']

    # handle division by zero
    df.loc[df['Volatility'] == 0, 'Sharpe_Ratio'] = 0

    # remove rows with NaN or inf returns
    df = df.dropna(subset=['Return', 'Sharpe_Ratio'])
    df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]

    return df

calculate_returns("XPHS:AC","2023-04-01","2023-04-10")

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- XPHS:AC: No timezone found, symbol may be delisted


/var/folders/s3/vv9psq2d7nb0zc29tlx4z90m0000gn/T/ipykernel_52254/1394361110.py:25: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]


,Open,High,Low,Close,Adj Close,Volume,Return,Volatility,Mean_Return,Sharpe_Ratio
Date,,,,,,,,,,


In [30]:
import yfinance as yf
import pandas as pd

ticker = "PSEI.PS"
start_date = "2015-01-01"
end_date = "2023-04-13"

data = yf.download(ticker, start=start_date, end=end_date)
data = data[['Close']]  # select only the 'Close' column
data_copy = data.copy()  # create a copy of the dataframe
data_copy['Date'] = data_copy.index  # add the 'Date' column

# specify the path to the folder where you want to save the CSV file
path = '/Users/jomarjordas/Documents/MSFIN299/MSFIN299-Research/_data/stockdata'

# save the CSV file to the specified folder
data_copy.to_csv(path + 'psei.csv', index=False)  # index=False to not include the index in the output file


[*********************100%***********************]  1 of 1 completed


In [49]:
import pandas as pd
import os
import statsmodels.api as sm

# Specify the directory where your files are located
data_dir = "/Users/jomarjordas/Documents/MSFIN299/MSFIN299-Research/_data/stockdata"

# Read in the index file
index_df = pd.read_csv(os.path.join(data_dir, 'psei.csv'))
index_df['date'] = pd.to_datetime(index_df['date'], format='%Y-%m-%d')

# Calculate the daily returns for the index
index_df['returns'] = index_df['price'].pct_change()

# Loop through each company file
for ticker in ['ac']:
    # Read in the company file
    company_df = pd.read_csv(os.path.join(data_dir, f'{ticker}.csv'))
    company_df['date'] = pd.to_datetime(company_df['date'], format='%Y-%m-%d')

    # Add this code to handle non-numeric values in the 'price' column
    returns = pd.to_numeric(company_df['price'], errors='coerce').pct_change()
    if returns.isna().any():
        company_df = company_df.dropna(subset=['price'])
        returns = pd.to_numeric(company_df['price'], errors='coerce').pct_change()
    company_df['returns'] = returns

    # Merge with index data
    merged_df = pd.merge(company_df, index_df, on='date', how='inner')

    # Check for and handle missing or invalid values in returns_y
    merged_df = merged_df[~merged_df['returns_y'].isna()]
    merged_df = merged_df[merged_df['returns_y'] != float('inf')]
    merged_df = merged_df[merged_df['returns_y'] != float('-inf')]

    # Calculate the excess returns
    merged_df['excess_return'] = merged_df['returns_x'] - merged_df['returns_y']

    # Calculate the expected return using CAPM
    X = sm.add_constant(merged_df['returns_y'])
    model = sm.OLS(merged_df['excess_return'], X)
    results = model.fit()
    beta = results.params[1]
    market_risk_premium = merged_df['returns_y'].mean() - 0.02 # Assume risk-free rate of 2%
    expected_return = 0.02 + beta * market_risk_premium

    # Add expected return column to company file
    merged_df['exp_return'] = expected_return

    # Calculate the abnormal returns
    merged_df['abnormal_return'] = merged_df['excess_return'] - merged_df['exp_return']

    # Calculate the cumulative abnormal returns
    merged_df['cumulative_abnormal_return'] = merged_df['abnormal_return'].cumsum()

    # Write merged dataframe back to company file
    merged_df.to_csv(os.path.join(data_dir, f'{ticker}.csv'), index=False)


KeyError: 'price'